In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from scipy.sparse import csr_matrix
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import confusion_matrix, classification_report

from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier

# NLP Project 2022

## Get data

In [ ]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/SakibShahriar95/ANTiVax/main/Labeled/VaxMisinfoData.csv'
df = pd.read_csv(url)
df.head()

,id,is_misinfo
0,1344795424855642112,0
1,1344794858133860353,0
2,1344794822691983360,0
3,1344794752819077123,1
4,1344792070507134977,0


In [ ]:
#  !pip install twarc
#  !twarc configure

Didn't work because of authentication

In [ ]:
'''
from twarc import Twarc
from urllib.request import urlretrieve

tweet_ids_url = 'https://raw.githubusercontent.com/SakibShahriar95/ANTiVax/main/Labeled/ids.txt'
tweet_ids = urlretrieve(tweet_ids_url, './ids.txt') # it's a file like object and works just like a file

df = pd.read_csv('./ids.txt')
df.head()

consumer_key="2lNba8Nvu9f0i5YgfmQhfh3mP"
consumer_secret="TGVv9QJhtkjSqm7QyMeLfpZm3IU3xjrrMabyae42VxbkaT5t0g"
access_token="1600730532-ZCeeHr9h033QRlC95BLZrpz85TRFN4pZ0w7vGhY"
access_token_secret="eoiufziPLBr0eYgR12gAoAEiTQjlnTkZ1BHBLXoDuQbtC"
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

df = pd.DataFrame()
for tweet in t.hydrate(open('./ids.txt')):
  row = { 'id': tweet['id'], 'text': tweet['text'] }
  df.append(row, ignore_index = True)
'''


'\nfrom twarc import Twarc\nfrom urllib.request import urlretrieve\n\ntweet_ids_url = \'https://raw.githubusercontent.com/SakibShahriar95/ANTiVax/main/Labeled/ids.txt\'\ntweet_ids = urlretrieve(tweet_ids_url, \'./ids.txt\') # it\'s a file like object and works just like a file\n\ndf = pd.read_csv(\'./ids.txt\')\ndf.head()\n\nconsumer_key="2lNba8Nvu9f0i5YgfmQhfh3mP"\nconsumer_secret="TGVv9QJhtkjSqm7QyMeLfpZm3IU3xjrrMabyae42VxbkaT5t0g"\naccess_token="1600730532-ZCeeHr9h033QRlC95BLZrpz85TRFN4pZ0w7vGhY"\naccess_token_secret="eoiufziPLBr0eYgR12gAoAEiTQjlnTkZ1BHBLXoDuQbtC"\nt = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)\n\ndf = pd.DataFrame()\nfor tweet in t.hydrate(open(\'./ids.txt\')):\n  row = { \'id\': tweet[\'id\'], \'text\': tweet[\'text\'] }\n  df.append(row, ignore_index = True)\n'

We hydrated offline and saved it to Github to retrieve it from there

In [ ]:
url = 'https://raw.githubusercontent.com/krautgortna/NLP_Project_SS22/main/data/hydrated_tweets.csv'
all_data = pd.read_csv(url)
all_data.text

0        My 87 year old grandmother has been quarantini...
1        I’m so excited that with this vaccine I can se...
2        So excited that I was able to close out 2020 o...
3        Well, it happened. My daughter used my grandch...
4        I’m getting my COVID-19 vaccine, baby! I’m so ...
                               ...                        
12292    Ooh! Potential #CoronaVac by @Sinovac side-eff...
12293    This rocks, very excited to get the vaccine as...
12294    We're excited to announce a COVID-19 vaccinati...
12295    Dec 2020:\nMOH: Vaccines are here!\nFPs: Yay! ...
12296    Fauci to #unvaccinated Americans: “get over th...
Name: text, Length: 12297, dtype: object

In [ ]:
Total = all_data.isnull().sum().sort_values(ascending=False)
percent = (all_data.isnull().sum() / all_data.isnull().count()).sort_values(ascending=False)
missing_stats = pd.concat([Total, percent], axis=1, keys=['Total', 'Percent'])

all_data_drop_80_missing = all_data.drop((missing_stats[missing_stats['Percent'] > .8]).index, axis=1)
missing_stats.head(25)

,Total,Percent
in_reply_to_status_id,12297,1.000000
retweet_screen_name,12297,1.000000
retweet_id,12297,1.000000
quote_id,12297,1.000000
in_reply_to_user_id,12297,1.000000
user_time_zone,12297,1.000000
in_reply_to_screen_name,12297,1.000000
coordinates,12265,0.997398
place,11785,0.958364
media,9704,0.789136


In [ ]:
df_merged = df.merge(all_data[['id', 'text', 'hashtags']], on='id')
df_merged


,id,is_misinfo,text,hashtags
0,1344795424855642112,0,"if you kissed someone on Amelia St, don’t worr...",NaN
1,1344794858133860353,0,i really doubt 2021 will be much different imo...,NaN
2,1344794822691983360,0,"People want answers, but they’re also just exc...",NaN
3,1344794752819077123,1,Why any healthy person would want to take an e...,NaN
4,1344790296119422980,1,1/1 @TheBMA surely consenting people to a two ...,NaN
...,...,...,...,...
12292,1413088663886573569,1,"Go ahead, take it...#Vaccine #TheJab #Genocide...",Vaccine TheJab Genocide Depopulation Resist198...
12293,1413087751474397186,0,Going to my first vaccine today. Very very ner...,firstjab VACCINE hull hull2021
12294,1413087030578401283,0,"Media: ""The #Japanese government declared a ne...",Japanese coronavirus
12295,1413085519710363648,0,Getting my first dose vaccine tomorrow. HUKM d...,NaN


In [ ]:
list(df_merged['hashtags'].str.split(" ").explode('hashtags').unique())

# EDA

## Preprocessing

In [ ]:
import re

def remove_targets(review_text):
    return re.sub('(@[A-Za-z0-9])\w+', '', review_text)

def remove_hashtags(review_text):
    return review_text.replace("#", " ")

def remove_numbers(review_text):
    return re.sub('[0-9]', '', review_text)

def remove_punctuation(review_text):
    return re.sub(r'[/\[\]\\\{\}\(\)\|\;\'\:\"\,\!\?\.]', '', review_text)

def remove_newlines(review_text):
    return review_text.replace("\n", " ")

df_cleaned = df_merged.copy(deep=True)

df_cleaned['text'] = df_cleaned['text'].apply(remove_targets)
df_cleaned['text'] = df_cleaned['text'].apply(remove_hashtags)
df_cleaned['text'] = df_cleaned['text'].apply(remove_numbers)

df_cleaned['text'] = df_cleaned['text'].apply(remove_punctuation)
df_cleaned['text'] = df_cleaned['text'].apply(remove_newlines)

df_cleaned['text'] = df_cleaned['text'].str.encode('ascii', 'ignore').str.decode('ascii')
df_cleaned['text'] = df_cleaned['text'].str.lower()

df_cleaned

,id,is_misinfo,text,hashtags
0,1344795424855642112,0,if you kissed someone on amelia st dont worry ...,NaN
1,1344794858133860353,0,i really doubt will be much different imo bec...,NaN
2,1344794822691983360,0,people want answers but theyre also just excit...,NaN
3,1344794752819077123,1,why any healthy person would want to take an e...,NaN
4,1344790296119422980,1,surely consenting people to a two part licen...,NaN
...,...,...,...,...
12292,1413088663886573569,1,go ahead take it vaccine thejab genocide de...,Vaccine TheJab Genocide Depopulation Resist198...
12293,1413087751474397186,0,going to my first vaccine today very very nerv...,firstjab VACCINE hull hull2021
12294,1413087030578401283,0,media the japanese government declared a new ...,Japanese coronavirus
12295,1413085519710363648,0,getting my first dose vaccine tomorrow hukm dr...,NaN


## Train model

In [ ]:
df_original = df.merge(all_data[['id', 'text']], on='id')
df_original

In [ ]:
X = df_original.drop('is_misinfo', axis=1)
y = df_original.is_misinfo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# Unigram Counts

X_train_texts = X_train.text

unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
unigram_vectorizer.fit(X_train_texts)
X_train_unigram = unigram_vectorizer.transform(X_train_texts)


# Unigram Tf-Idf

unigram_tf_idf_transformer = TfidfTransformer()
unigram_tf_idf_transformer.fit(X_train_unigram)

X_train_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_train_unigram)


# Bigram Counts

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
bigram_vectorizer.fit(X_train_texts)
X_train_bigram = bigram_vectorizer.transform(X_train_texts)


# Bigram Tf-Idf

bigram_tf_idf_transformer = TfidfTransformer()
bigram_tf_idf_transformer.fit(X_train_bigram)

X_train_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_train_bigram)

In [ ]:
def train_and_show_scores(X: csr_matrix, y: np.array, title: str) -> None:
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, train_size=0.8, stratify=y
    )

    clf = SGDClassifier()
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    valid_score = clf.score(X_valid, y_valid)
    print(f'{title}\nSGDClassifier: Train score: {round(train_score, 2)} ; Validation score: {round(valid_score, 2)}')

    clf = XGBClassifier()
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    valid_score = clf.score(X_valid, y_valid)
    print(f'XGBoostClassifier: Train score: {round(train_score, 2)} ; Validation score: {round(valid_score, 2)}')

    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    valid_score = clf.score(X_valid, y_valid)
    print(f'MultinomialNB: Train score: {round(train_score, 2)} ; Validation score: {round(valid_score, 2)}\n')

train_and_show_scores(X_train_unigram, y_train, 'Unigram Counts')
train_and_show_scores(X_train_unigram_tf_idf, y_train, 'Unigram Tf-Idf')
train_and_show_scores(X_train_bigram, y_train, 'Bigram Counts')
train_and_show_scores(X_train_bigram_tf_idf, y_train, 'Bigram Tf-Idf')

In [ ]:
x_train_transformed = X_train_bigram_tf_idf #this had the best validation score (88%)
clf = SGDClassifier()

distributions = dict(
    penalty=['l1', 'l2', 'elasticnet'],
    alpha=uniform(loc=1e-6, scale=1e-4),
    loss=['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    learning_rate=['optimal', 'invscaling', 'adaptive'],
    eta0=uniform(loc=1e-7, scale=1e-2)
)

random_search_cv = RandomizedSearchCV(
    estimator=clf,
    param_distributions=distributions,
    cv=5,
    n_iter=50,
    n_jobs = -1,
)
random_search_cv.fit(x_train_transformed, y_train)
print(f'Best params: {random_search_cv.best_params_}')
print(f'Best score: {random_search_cv.best_score_}')

In [ ]:
sgd_classifier = random_search_cv.best_estimator_

#Best params: {'alpha': 9.232722822313204e-05, 'eta0': 0.006418957642841877, 'learning_rate': 'adaptive', 'loss': 'squared_hinge', 'penalty': 'l2'}
sgd_classifier = SGDClassifier(alpha=9.232722822313204e-05, eta0=0.006418957642841877, learning_rate='adaptive', loss='squared_hinge', penalty='l2')
sgd_classifier.fit(x_train_transformed, y_train)

#transform test data
x_test_transformed = bigram_vectorizer.transform(X_test.text)
x_test_transformed = bigram_tf_idf_transformer.transform(x_test_transformed)

#test score
score = sgd_classifier.score(x_test_transformed, y_test)
print("Test score SGD: ", score)

# try MultinomialNB

# Vectorize text reviews to numbers
vec = CountVectorizer(ngram_range=(1, 1))
x_train = vec.fit_transform(X_train_texts).toarray()
x_test = vec.transform(X_test.text).toarray()

model = MultinomialNB()
model.fit(x_train, y_train)
score = model.score(x_test, y_test)
print("Test score MultinomialNB: ", score)

In [ ]:
clf = XGBClassifier()
clf.fit(x_train_transformed, y_train)
score = clf.score(x_test_transformed, y_test)
print("Test score XGB: ", score)

## Tuning

## Debugging

In [ ]:
y_pred = sgd_classifier.predict(x_test_transformed)

print("Confusion Matrix")
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)

ax = sns.heatmap(cf_matrix, annot=True, fmt='.0f', cmap='Blues')

ax.set_title('Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ')

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Negative','Positive'])
ax.yaxis.set_ticklabels(['Negative','Positive'])

## Display the visualization of the Confusion Matrix.
plt.show()

# Classification Report
print("\nClassification Report")
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
X_test['is_misinfo_pred'] = y_pred
X_test['is_misinfo_true'] = y_test
X_test.loc[~(X_test['is_misinfo_pred'] == X_test['is_misinfo_true'])]